## Importing the Dataset and Libraries

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import metrics

In [2]:
data = pd.read_csv('Train_Data.csv')

In [3]:
data.head()

,Title,Date,Abstract,Subjects
0,The Neutrino Ball Model of a Quasar,1993-04-07,It is suggested that the nonorthodox model of a quasar as a neutrino ball\ndescribed in terms of the standard model extended by adding right-handed\nneutrinos and the Majorana scalar field can be presented in order to explain a\nquasar as a body of weak interacting neutrinos. Neutrino interaction with the\nscalar Majorana field violates the lepton number and produces the mass\nsplitting of the neutrino due to the sea-saw mechanism. In this model a quasar\nis an object which appears in the result of the first order cosmological phase\ntransition. In this interpretation a quasar may be regarded as a ball filled\nwith Dirac neutrinos and can be treated as a remnant of phase transition with\nunbroken global lepton symmetry. In this paper we study the macroscopic\nparameters of such a configuration. In the result the mass-radius curve M(R)\nfor the quasar is obtained.\n,['Astrophysics']
1,Charged Boson Stars and Vacuum Instabilities,1993-05-12,"We consider charged boson stars and study their effect on the structure of\nthe vacuum. For very compact particle like ``stars"", with constituent mass\n$m_{*}$ close to the Planck mass $m_{Pl}$, i.e. $m_{*}^{2} = {\\cal O} (\\alpha\nm_{Pl}^{2})$, we argue that there is a limiting total electric charge $Z_c$,\nwhich, primarily, is due to the formation of a pion condensate ($Z_{c} \\simeq\n0.5\\alpha^{-1}e$, where $\\alpha$ is the fine structure constant and $e$ is the\nelectric charge of the positron). If the charge of the ``star"" is larger than\n$Z_c$ we find numerical evidence for a complete screening indicating a limiting\ncharge for a very compact object.\n There is also a less efficient competing charge screening mechanism due to\nspontaneous electron-positron pair creation in which case $Z_{c} \\simeq\n\\alpha^{-1}e$. Astrophysical and cosmological abundances of charged compact\nboson stars are briefly discussed in terms of dark matter.\n","['Astrophysics', 'General Relativity and Quantum Cosmology']"
2,Pre-main sequence lithium burning. I. Weak T Tauri stars,1993-08-31,"We derive lithium abundances in 53 T Tauri stars (TTS), concentrating on\nweak-line TTS. Our study gives the following results: 1)At luminosities $\\ge\n0.9 L\\sun$ the Li abundances are remarkably uniform, with a mean value,\nlog~N(Li)=3.1, equal to the ""cosmic"" lithium abundance. 2)Significant Li\ndepletion appears below 0.5~L$\\sun$ in the mass range 0.9-0.2 M$\\sun$ and\nincreases towards lower luminosities. At the lower mass end (0.4-0.2 M$\\sun$),\nthe observed luminosity of the Li burning turning point is about a factor 4\nhigher than predicted by the models. At masses 1.2-1.0 M$\\sun$ the observations\nimply less PMS Li burning than theoretically expected. Moreover we show that:\n1)Low Li abundances appear only among stars with low $v\\sin{i}$. Fast rotators\nwith masses around 0.8~M$\\sun$ do not show evidence for strong Li depletion\ntowards lower luminosities as slow rotators do. 2)In a sample restricted to\nonly K5-K7 stars we find that the angular momentum spread before Li burning\nbegins is larger than a factor 10. Lithium depletion associated to angular\nmomentum loss during PMS evolution is not required to explain the observed\nabundances. The efficiency of PMS Li burning in the mass range 0.9-0.7~M$\\sun$\nis reduced in the presence of rapid rotation.\n",['Astrophysics']
3,Very High Energy Gamma-Rays from AGN: Cascading on the Cosmic Background\n Radiation Fields and the Formation of Pair Halos,1993-12-18,"Recent high energy gamma-ray observations (E>100 MeV) of blazar AGN show\nemission spectra with no clear upper energy cutoff. AGN, considered to be\npossible sources for the highest energy cosmic rays, may have emission\nextending well into the VHE (very high energy, E>100 GeV) domain. Because VHE\ngamma-rays are absorbed by pair production on the intergalactic background\nradiation, much of this emission is not visible

## Data PreProcessing

In [4]:
data.drop(['Date'],axis=1,inplace=True)
data['Subjects']=data['Subjects'].str.replace('[','')
data['Subjects']=data['Subjects'].str.replace(']','')
data1 = data['Subjects'].str.split(',',expand=True)
data['Subjects'] = data1[0]

In [5]:
title =[]
subjects =[]
abstract=[]
for i in range(1,8):
    for j in range(len(data1[i])):
        if(data1[i][j]!=None):
            title.append(data['Title'][j])
            subjects.append(data['Subjects'][j])
            abstract.append(data['Abstract'][j])
data_join=pd.DataFrame(list(zip(title, abstract,subjects)), 
               columns =['Title','Abstract','Subjects']) 

In [6]:
df = pd.concat([data,data_join],ignore_index=True)

In [ ]:
# Considering both Title and Abstract for training

In [79]:
X =df['Title'].str.cat(df["Abstract"],sep=" ")
y =df['Subjects']

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [115]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(31904, 64077)

In [116]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(31904, 64077)

In [83]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', RandomForestClassifier(n_estimators=150)),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                               

In [123]:
predictions = text_clf.predict(X_test)

In [ ]:
# Accuracy of the model is 88.3%

In [86]:
print(metrics.accuracy_score(y_test,predictions))

0.8834871142220808


## Filling the values in submit.csv

In [87]:
test_df=pd.read_csv("Test_Data.csv")

In [88]:
date_backup=test_df["Date"]

In [89]:
test_df.drop(["Date"],axis=1,inplace=True)

In [98]:
testing_data =test_df['Title'].str.cat(test_df["Abstract"],sep=" ")

In [99]:
predictions=text_clf.predict(testing_data)

In [100]:
predictions

array(["'Astrophysics - High Energy Astrophysical Phenomena'",
       "'Astrophysics - Cosmology and Nongalactic Astrophysics'",
       "'Astrophysics - Solar and Stellar Astrophysics'", ...,
       "'Astrophysics - High Energy Astrophysical Phenomena'",
       "'Astrophysics - High Energy Astrophysical Phenomena'",
       "'Astrophysics - Solar and Stellar Astrophysics'"], dtype=object)

In [101]:
pred_df=pd.DataFrame(predictions,columns=["Prediction"])

In [102]:
Date_df=date_backup.str.split("-",expand=True)
Date_df.rename(columns={0:"Year",1:"Month",2:"DATE"},inplace=True)

In [103]:
a=Date_df["Month"]

In [104]:
submit_df=pd.concat((a,pred_df),axis=1)

In [105]:
may=submit_df[submit_df["Month"]=='05']
may["Prediction"].value_counts()

'Astrophysics - Astrophysics of Galaxies'                        379
'Astrophysics - High Energy Astrophysical Phenomena'             290
'Astrophysics - Solar and Stellar Astrophysics'                  205
'Astrophysics - Cosmology and Nongalactic Astrophysics'          159
'Astrophysics - Earth and Planetary Astrophysics'                135
'Astrophysics - Instrumentation and Methods for Astrophysics'    93 
'General Relativity and Quantum Cosmology'                       72 
'High Energy Physics - Phenomenology'                            36 
'Nuclear Theory'                                                 1  
'High Energy Physics - Theory'                                   1  
Name: Prediction, dtype: int64

In [106]:
may=submit_df[submit_df["Month"]=='06']
may["Prediction"].value_counts()

'Astrophysics - Astrophysics of Galaxies'                        321
'Astrophysics - High Energy Astrophysical Phenomena'             265
'Astrophysics - Solar and Stellar Astrophysics'                  208
'Astrophysics - Cosmology and Nongalactic Astrophysics'          147
'Astrophysics - Earth and Planetary Astrophysics'                113
'Astrophysics - Instrumentation and Methods for Astrophysics'    76 
'General Relativity and Quantum Cosmology'                       61 
'High Energy Physics - Phenomenology'                            27 
'High Energy Physics - Theory'                                   1  
'Nuclear Theory'                                                 1  
Name: Prediction, dtype: int64

In [107]:
may=submit_df[submit_df["Month"]=='07']
may["Prediction"].value_counts()

'Astrophysics - Astrophysics of Galaxies'                        357
'Astrophysics - High Energy Astrophysical Phenomena'             290
'Astrophysics - Cosmology and Nongalactic Astrophysics'          214
'Astrophysics - Solar and Stellar Astrophysics'                  196
'Astrophysics - Instrumentation and Methods for Astrophysics'    177
'Astrophysics - Earth and Planetary Astrophysics'                139
'General Relativity and Quantum Cosmology'                       55 
'High Energy Physics - Phenomenology'                            32 
Name: Prediction, dtype: int64

In [108]:
may=submit_df[submit_df["Month"]=='08']
may["Prediction"].value_counts()

'Astrophysics - High Energy Astrophysical Phenomena'             345
'Astrophysics - Astrophysics of Galaxies'                        324
'Astrophysics - Solar and Stellar Astrophysics'                  236
'Astrophysics - Instrumentation and Methods for Astrophysics'    150
'Astrophysics - Cosmology and Nongalactic Astrophysics'          143
'Astrophysics - Earth and Planetary Astrophysics'                129
'General Relativity and Quantum Cosmology'                       60 
'High Energy Physics - Phenomenology'                            19 
Name: Prediction, dtype: int64

In [109]:
may=submit_df[submit_df["Month"]=='09']
may["Prediction"].value_counts()

'Astrophysics - Astrophysics of Galaxies'                        433
'Astrophysics - High Energy Astrophysical Phenomena'             341
'Astrophysics - Solar and Stellar Astrophysics'                  219
'Astrophysics - Cosmology and Nongalactic Astrophysics'          161
'Astrophysics - Instrumentation and Methods for Astrophysics'    147
'Astrophysics - Earth and Planetary Astrophysics'                126
'General Relativity and Quantum Cosmology'                       61 
'High Energy Physics - Phenomenology'                            24 
Name: Prediction, dtype: int64

In [110]:
may=submit_df[submit_df["Month"]=='10']
may["Prediction"].value_counts()

'Astrophysics - Astrophysics of Galaxies'                        102
'Astrophysics - High Energy Astrophysical Phenomena'             78 
'Astrophysics - Solar and Stellar Astrophysics'                  75 
'Astrophysics - Cosmology and Nongalactic Astrophysics'          60 
'Astrophysics - Earth and Planetary Astrophysics'                32 
'Astrophysics - Instrumentation and Methods for Astrophysics'    32 
'General Relativity and Quantum Cosmology'                       15 
'High Energy Physics - Phenomenology'                            7  
Name: Prediction, dtype: int64